In [67]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split



In [23]:


# Transformers
from transformers import BartTokenizer, BartForConditionalGeneration      # BERT Tokenizer and architecture
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments         # These will help us to fine-tune our model
from transformers import pipeline                                         # Pipeline
from transformers import DataCollatorForSeq2Seq                           # DataCollator to batch the data
import torch                                                              # PyTorch
# import evaluate                                                            # Evaluation metric

# Other NLP libraries
from textblob import TextBlob                                             # This is going to help us fix spelling mistakes in texts
from sklearn.feature_extraction.text import TfidfVectorizer               # This is going to helps identify the most common terms in the corpus
import re                                                                 # This library allows us to clean text data
import nltk                                                               # Natural Language Toolkit
nltk.download('punkt')                                                    # This divides a text into a list of sentences

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Load data

In [24]:
pd.set_option('display.max_colwidth', 1000) #this is to allow us to show full text

In [25]:
df = pd.read_csv('cp24_20.csv')

In [26]:
df.shape

(21, 11)

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   21 non-null     int64 
 1   text         21 non-null     object
 2   text_length  21 non-null     int64 
 3   total_word   21 non-null     int64 
 4   summary      21 non-null     object
 5   min_word     21 non-null     int64 
 6   max_word     21 non-null     int64 
 7   source       21 non-null     object
 8   url          21 non-null     object
 9   type         21 non-null     object
 10  model_used   21 non-null     object
dtypes: int64(5), object(6)
memory usage: 1.9+ KB


In [28]:
df.head()

,Unnamed: 0,text,text_length,total_word,summary,min_word,max_word,source,url,type,model_used
0,0,"Police are investigating multiple reports of falling objects striking and damaging vehicles near Toronto's Don Valley Parkway in recent months. The Toronto Police Service (TPS) said it first received reports that two vehicles had been damaged by falling objects in the area of Pottery Road and Broadview Avenue in March. Officers determined that the vehicles had been damaged by “various sizes of metal BBs and glass marbles,” police said in a release issued Thursday. Police said they do not know where the objects originated from or who may be responsible. Police said they received a third report in July of a vehicle in the same area damaged by falling objects. In that incident, they said an unknown person threw glass, ball bearings and marbles onto vehicles on Pottery Road. “Drivers, pedestrians, and cyclists in the area are urged to use caution and to be vigilant as police continue to investigate,” the release states. Investigators asked anyone with information to call 416-808-5500,...",1062,170,Police are investigating multiple reports of falling objects striking and damaging vehicles near Toronto's Don Valley Parkway . The Toronto Police Service said it first received reports that two vehicles had been damaged by falling objects in March . Police said they do,37,47,cp24,https://www.cp24.com/news/police-investigation-reports-of-falling-objects-damaging-vehicles-near-the-don-valley-parkway-1.6969051,news,summarizer
1,1,"A Toronto woman is looking to recover the specialized bicycle used to raise money for cancer research after it was stolen from her Etobicoke home over the weekend. Monica Doedens said she returned home on Saturday to find the lock on her door broken, her house ransacked and several valuable items stolen. The hardest loss to accept was her Pinarello bike, stored in the basement — purchased three years ago. “I could feel my heart drop,” Doedens told CTV News Toronto. “We have a garage and a shed, and I deliberately do not keep [the bike] in either because I thought the house was safest.” “The bicycle was an expensive one which I really didn’t think it could afford but because I ride so much I decided to spend the extra money on this bike and it was a little bit heartbreaking,” she continued. Doedens said the bike was custom-made and will cost $10,000 to replace. She said she doesn’t know yet if her home insurance will cover the loss. “They don’t make the Pinarello Prince anymore,” s...",1965,355,"Monica Doedens returned home on Saturday to find the lock on her door broken and her house ransacked . The hardest loss to accept was her custom-made bike, purchased three years ago . The bike will cost $10,000 to replace and she's raising money for cancer research . Toronto Police stats show about 3,000 bikes are stolen a year, most were taken from outside residences . Doeden said she doesn't know yet if",83,93,cp24,https://www.cp24.com/news/heartbreaking-bike-used-for-cancer-fundraising-stolen-from-toronto-woman-s-basement-1.6975085,news,summarizer
2,2,"The brother of a New Zealand man who died after ordering a product from an accused Canadian suicide salesman says it shouldn’t be so easy for vulnerable people to get dangerous substances online. Sean Plunket says he’s shocked at how easily his brother, James, ordered the fatal package from Canada, and has tough questions for this country’s internet regulators and e-commerce platforms about why they didn't stop it. “The idea he could access this, not necessarily through the dark web, but it was there, hiding in plain bloody sight, that is chilling to me,” Plunket said in an interview from New Zealand. “We know he was vulnerable at that moment. At that moment he needed protection, there was someone who took advantage of him and made it easier for him to take his own life rather than harder,” he said. “Those bad decisions become very, very easy when you’ve got someone making mo

#Train and test

In [29]:
X_train,X_test,y_train,y_test = train_test_split(df['text'],df['summary'],test_size=0.1, shuffle=True, random_state=30)

In [30]:
print("X_train shape", X_train.shape)
print("X_test shape", X_test.shape)
print("y_train shape",y_train.shape)
print("y_test shape",y_test.shape)

X_train shape (18,)
X_test shape (3,)
y_train shape (18,)
y_test shape (3,)


In [31]:
X_train

14     A two-vehicle crash in Hamilton Tuesday night has left one person dead and three others injured, police say. Emergency crews were called to the area of Dundas Street East and Avonsyde Boulevard , near the city’s Burlington border, just after 11 p.m. for reports of a collision. Police said that’s where the 42-year-old driver of a black Dodge Ram and the three occupants of a white Toyota collided. The driver of the Dodge Ram was transported to hospital, where they were pronounced deceased, police said. The occupants of the Toyota sustained minor injuries and were transported to hospital. The cause of the crash is unknown at this time. Investigators with the Hamilton police collision reconstruction unit are on scene investigating. Road closures are in effect. Police are asking anyone with information about the crash to contact police at 905-546-4753 or via email at reconunit@hamiltonpolice.ca. . @HamiltonPolice are currently investigating a two-vehicle collision on Dundas Street Ea

In [63]:
X_test

17                                                                                                                                                                       The Office of the Privacy Commissioner of Canada says it’s opened an investigation into allegations that some Loblaw customers have been unable to delete their PC Optimum accounts. Spokesman Vito Pilieci said in an email that the office has received several such complaints. He said the office can’t comment further due to the active investigation. Loblaw spokeswoman Catherine Thomas said in an email that the company has processes to respond to account deletion requests in a timely manner, and that it will fully co-operate with the privacy commissioner's office. The Office of the Privacy Commissioner oversees compliance with the Personal Information Protection and Electronic Documents Act as well as the Privacy Act. According to Loblaw's 2023 annual report, the PC Optimum loyalty program has more than 16 million active us

#Modeling

In [86]:
#transformers
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.utils.data import DataLoader, Dataset

#vocabulary
from collections import Counter
import itertools

## Vocabulary creation

In [88]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   21 non-null     int64 
 1   text         21 non-null     object
 2   text_length  21 non-null     int64 
 3   total_word   21 non-null     int64 
 4   summary      21 non-null     object
 5   min_word     21 non-null     int64 
 6   max_word     21 non-null     int64 
 7   source       21 non-null     object
 8   url          21 non-null     object
 9   type         21 non-null     object
 10  model_used   21 non-null     object
dtypes: int64(5), object(6)
memory usage: 1.9+ KB


In [89]:
df.head(1)

,Unnamed: 0,text,text_length,total_word,summary,min_word,max_word,source,url,type,model_used
0,0,"Police are investigating multiple reports of falling objects striking and damaging vehicles near Toronto's Don Valley Parkway in recent months. The Toronto Police Service (TPS) said it first received reports that two vehicles had been damaged by falling objects in the area of Pottery Road and Broadview Avenue in March. Officers determined that the vehicles had been damaged by “various sizes of metal BBs and glass marbles,” police said in a release issued Thursday. Police said they do not know where the objects originated from or who may be responsible. Police said they received a third report in July of a vehicle in the same area damaged by falling objects. In that incident, they said an unknown person threw glass, ball bearings and marbles onto vehicles on Pottery Road. “Drivers, pedestrians, and cyclists in the area are urged to use caution and to be vigilant as police continue to investigate,” the release states. Investigators asked anyone with information to call 416-808-5500,...",1062,170,Police are investigating multiple reports of falling objects striking and damaging vehicles near Toronto's Don Valley Parkway . The Toronto Police Service said it first received reports that two vehicles had been damaged by falling objects in March . Police said they do,37,47,cp24,https://www.cp24.com/news/police-investigation-reports-of-falling-objects-damaging-vehicles-near-the-don-valley-parkway-1.6969051,news,summarizer


In [137]:

# Combine texts and summaries for vocabulary creation
textSummaryCombined = list(df['text']) + list(df['summary'])

#Tokenize the texts
def tokenize(text):
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    return text.lower().split()

#Count token frequencies
token_counter = Counter(itertools.chain(*map(tokenize, textSummaryCombined)))

#Create the vocabulary with special tokens
special_tokens = ["<PAD>", "<UNK>"]
most_common_tokens = [token for token, _ in token_counter.most_common()]
vocab = {token: idx for idx, token in enumerate(special_tokens + most_common_tokens)}

# Optionally limit the vocabulary size
vocab_size = 10000  # or any desired size
if len(vocab) > vocab_size:
    vocab = {k: v for k, v in itertools.islice(vocab.items(), vocab_size)}

# Display the vocabulary. vocab = word : ID
print(vocab)


{'<PAD>': 0, '<UNK>': 1, 'alert': 2, 'the': 3, 'to': 4, 'a': 5, 'in': 6, 'of': 7, 'and': 8, 'was': 9, 'said': 10, 'on': 11, 'for': 12, 'that': 13, 'is': 14, 'police': 15, 'at': 16, 'are': 17, 'with': 18, 'he': 19, 'has': 20, 'it': 21, 'as': 22, 'be': 23, 'i': 24, 'after': 25, 'from': 26, 'were': 27, 'toronto': 28, 'have': 29, 'they': 30, 'who': 31, 'an': 32, 'by': 33, 'been': 34, 'not': 35, 'she': 36, 'his': 37, 'canada': 38, 'this': 39, 'but': 40, 'news': 41, 'more': 42, 'will': 43, 'if': 44, 'its': 45, 'two': 46, 'ctv': 47, 'their': 48, 'her': 49, 'we': 50, 'about': 51, 'one': 52, 'tuesday': 53, 'just': 54, 'man': 55, 'says': 56, 'canadian': 57, 'you': 58, 'or': 59, 'people': 60, 'being': 61, 'harris': 62, 'so': 63, 'time': 64, 'up': 65, 'when': 66, 'city': 67, 'charges': 68, 'say': 69, 'bank': 70, 'cuts': 71, 'interest': 72, 'rate': 73, 'inflation': 74, 'near': 75, 'there': 76, 'also': 77, 'work': 78, 'traffic': 79, 'first': 80, 'area': 81, 'new': 82, 'get': 83, 'suspect': 84, 'day'

In [138]:
# Save the vocabulary to a text file
vocab_file_path = 'vocab.txt'

with open(vocab_file_path, 'w') as f:
    for word, idx in vocab.items():
        f.write(f"{word}\t{idx}\n")

print(f"Vocabulary saved to {vocab_file_path}")


Vocabulary saved to vocab.txt


In [157]:


class TransformerBlock(nn.Module):
    def __init__(self, embed_size, heads, dropout):
        super(TransformerBlock, self).__init__()
        self.attention = nn.MultiheadAttention(embed_size, heads)
        self.norm1 = nn.LayerNorm(embed_size)
        self.norm2 = nn.LayerNorm(embed_size)
        self.ff = nn.Sequential(
            nn.Linear(embed_size, embed_size * 4),
            nn.ReLU(),
            nn.Linear(embed_size * 4, embed_size)
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        attn_output, _ = self.attention(x, x, x)
        x = self.norm1(attn_output + x)
        ff_output = self.ff(x)
        out = self.norm2(ff_output + x)
        return out

class SummarizerModel(nn.Module):
    def __init__(self, embed_size, heads, num_layers, vocab_size, max_len, dropout):
        super(SummarizerModel, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.pos_embedding = nn.Embedding(max_len, embed_size)
        self.layers = nn.ModuleList(
            [TransformerBlock(embed_size, heads, dropout) for _ in range(num_layers)]
        )
        self.fc_out = nn.Linear(embed_size, vocab_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        N, seq_length = x.shape
        positions = torch.arange(0, seq_length).expand(N, seq_length).to(x.device)
        out = self.dropout(self.embed(x) + self.pos_embedding(positions))
        for layer in self.layers:
            out = layer(out)
        out = self.fc_out(out)
        return out

# Hyperparameters
embed_size = 256
heads = 8
num_layers = 6
vocab_size = len(vocab)  # Example vocab size
max_len = 512
dropout = 0.1

model = SummarizerModel(embed_size, heads, num_layers, vocab_size, max_len, dropout)


In [158]:


class TextSummaryDataset(Dataset):
    def __init__(self, texts, summaries, tokenizer, max_len):
        self.texts = texts
        self.summaries = summaries
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        summary = self.summaries[idx]
        input_ids = self.tokenizer(text, max_length=self.max_len, padding='max_length', truncation=True)
        summary_ids = self.tokenizer(summary, max_length=self.max_len, padding='max_length', truncation=True)
        return torch.tensor(input_ids), torch.tensor(summary_ids)

# Example tokenizer
# def simple_tokenizer(text, max_length):
#     tokens = text.split()[:max_length]
#     return [vocab.get(token, vocab['<UNK>']) for token in tokens] + [0] * (max_length - len(tokens))

def simple_tokenizer(text, max_length, padding='max_length', truncation=True):
    tokens = text.split()
    if truncation and len(tokens) > max_length:
        tokens = tokens[:max_length]
    token_ids = [vocab.get(token, vocab['<UNK>']) for token in tokens]

    if padding == 'max_length' and len(token_ids) < max_length:
        token_ids += [vocab['<PAD>']] * (max_length - len(token_ids))

    print(f"Token IDs: {token_ids}")
    return token_ids


In [159]:
len(vocab)

2581

In [160]:
list(vocab.keys())[:20]

['<PAD>',
 '<UNK>',
 'alert',
 'the',
 'to',
 'a',
 'in',
 'of',
 'and',
 'was',
 'said',
 'on',
 'for',
 'that',
 'is',
 'police',
 'at',
 'are',
 'with',
 'he']

In [161]:

texts = X_train[:2]
summaries = y_train[:2]

In [162]:
texts =list(texts)

In [163]:
summaries =list(summaries)

In [164]:
print(len(texts[0]),texts[0])

1339  A two-vehicle crash in Hamilton Tuesday night has left one person dead and three others injured, police say. Emergency crews were called to the area of Dundas Street East and Avonsyde Boulevard , near the city’s Burlington border, just after 11 p.m. for reports of a collision. Police said that’s where the 42-year-old driver of a black Dodge Ram and the three occupants of a white Toyota collided. The driver of the Dodge Ram was transported to hospital, where they were pronounced deceased, police said. The occupants of the Toyota sustained minor injuries and were transported to hospital. The cause of the crash is unknown at this time. Investigators with the Hamilton police collision reconstruction unit are on scene investigating. Road closures are in effect. Police are asking anyone with information about the crash to contact police at 905-546-4753 or via email at reconunit@hamiltonpolice.ca. . @HamiltonPolice are currently investigating a two-vehicle collision on Dundas Street Eas

In [165]:


dataset = TextSummaryDataset(texts, summaries, simple_tokenizer, max_len)
dataloader = DataLoader(dataset, batch_size=2)

optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)
criterion = nn.CrossEntropyLoss()

for epoch in range(3):  # Number of epochs
    model.train()
    for batch in dataloader:
        input_ids, summary_ids = batch

        optimizer.zero_grad()
        output = model(input_ids)
        loss = criterion(output.view(-1, vocab_size), summary_ids.view(-1))
        loss.backward()
        optimizer.step()
        print(f"Epoch: {epoch}, Loss: {loss.item()}")

Token IDs: [1, 1, 168, 6, 1, 1, 134, 20, 181, 52, 126, 86, 8, 127, 287, 1, 15, 1, 1, 684, 27, 178, 4, 3, 81, 7, 1, 1, 1, 8, 1, 1, 1, 75, 3, 1, 1, 1, 54, 25, 1242, 1, 12, 289, 7, 5, 1, 1, 10, 1, 124, 3, 1, 501, 7, 5, 1119, 1, 1, 8, 3, 127, 1244, 7, 5, 1010, 1, 1, 1, 501, 7, 3, 1, 1, 9, 411, 4, 1, 124, 30, 27, 659, 1, 15, 1, 1, 1244, 7, 3, 1, 2267, 2268, 188, 8, 27, 411, 4, 1, 1, 2269, 7, 3, 168, 14, 451, 16, 39, 1, 1, 18, 3, 1, 15, 331, 1246, 696, 17, 11, 148, 1, 1, 358, 17, 6, 1, 1, 17, 2270, 296, 18, 171, 51, 3, 168, 4, 246, 15, 16, 1, 59, 2272, 581, 16, 1, 1, 1, 17, 634, 242, 5, 1, 331, 11, 1, 1, 1, 8, 1, 1, 54, 25, 1, 1, 105, 1, 1, 2276, 502, 6, 953, 2277, 8, 14, 2278, 4, 2279, 1185, 2280, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 149, 1, 1, 126, 14, 86, 8, 127, 287, 17, 226, 25, 5, 1, 168, 6, 1, 11, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [172]:
def summarize(text, model, tokenizer, max_len=512):
    model.eval()
    input_ids = tokenizer(text, max_length=max_len, padding='max_length', truncation=True)
    print(f"Input IDs: {input_ids}")

    input_ids = torch.tensor(input_ids).unsqueeze(0)  # Add batch dimension
    print(f"Input IDs: {input_ids}")



    with torch.no_grad():
        output = model(input_ids)
        print(f"Output: {output}")

    summary_ids = output.argmax(dim=-1).squeeze().tolist()
    print(f"Summary IDs: {summary_ids}")  # Print the summary IDs
    summary = " ".join([vocab_inv[idx] for idx in summary_ids if idx not in [0]])  # Convert token IDs to words, ignoring padding
    return summary

# vocab inver = id : word
vocab_inv = {idx: word for word, idx in vocab.items()}
print(f"vocab inver",vocab_inv)

# Example summarization
text = " A two-vehicle crash in Hamilton Tuesday night has left one person dead and three others injured, police say. Emergency crews were called to the area of Dundas Street East and Avonsyde Boulevard , near the city’s Burlington border, just after 11 p.m. for reports of a collision. Police said that’s where the 42-year-old driver of a black Dodge Ram and the three occupants of a white Toyota collided. The driver of the Dodge Ram was transported to hospital, where they were pronounced deceased, police said. The occupants of the Toyota sustained minor injuries and were transported to hospital."
summary = summarize(text, model, simple_tokenizer, int(len(text)/2))
print(len(summary),summary)


vocab inver {0: '<PAD>', 1: '<UNK>', 2: 'alert', 3: 'the', 4: 'to', 5: 'a', 6: 'in', 7: 'of', 8: 'and', 9: 'was', 10: 'said', 11: 'on', 12: 'for', 13: 'that', 14: 'is', 15: 'police', 16: 'at', 17: 'are', 18: 'with', 19: 'he', 20: 'has', 21: 'it', 22: 'as', 23: 'be', 24: 'i', 25: 'after', 26: 'from', 27: 'were', 28: 'toronto', 29: 'have', 30: 'they', 31: 'who', 32: 'an', 33: 'by', 34: 'been', 35: 'not', 36: 'she', 37: 'his', 38: 'canada', 39: 'this', 40: 'but', 41: 'news', 42: 'more', 43: 'will', 44: 'if', 45: 'its', 46: 'two', 47: 'ctv', 48: 'their', 49: 'her', 50: 'we', 51: 'about', 52: 'one', 53: 'tuesday', 54: 'just', 55: 'man', 56: 'says', 57: 'canadian', 58: 'you', 59: 'or', 60: 'people', 61: 'being', 62: 'harris', 63: 'so', 64: 'time', 65: 'up', 66: 'when', 67: 'city', 68: 'charges', 69: 'say', 70: 'bank', 71: 'cuts', 72: 'interest', 73: 'rate', 74: 'inflation', 75: 'near', 76: 'there', 77: 'also', 78: 'work', 79: 'traffic', 80: 'first', 81: 'area', 82: 'new', 83: 'get', 84: 'sus

In [ ]:
#from dataset
#remove punct
#remove numbers

#work on the following
#fine tune the model, right now this is happening, there is not sense at all with text and summary
text
A two-vehicle crash in Hamilton Tuesday night has left one person dead and three others injured, police say. Emergency crews were called to the area of Dundas Street East and Avonsyde Boulevard , near the city’s Burlington border, just after 11 p.m. for reports of a collision. Police said that’s where the 42-year-old driver of a black Dodge Ram and the three occupants of a white Toyota collided. The driver of the Dodge Ram was transported to hospital, where they were pronounced deceased, police said. The occupants of the Toyota sustained minor injuries and were transported to hospital."

summary
focus olympic convicted southwestern ramara leaning main regina fifth footsteps hockey order generous stages came cost loblaw sent addison modification brother commute college struck confidence committing another endorsement rcmp jet premium

#increase the data of model

In [128]:
import torch

# Example model output
output = torch.randn(1, 10, 10000)  # Random output for illustration

# Extract the predicted token IDs
summary_ids = output.argmax(dim=-1).squeeze().tolist()

# Display the predicted token IDs
print(summary_ids)


[5774, 7249, 7876, 4922, 7326, 4582, 7752, 8920, 6557, 5271]


In [131]:
import re
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from collections import Counter
import itertools

# Example texts and summaries
texts = ["This is a sample document.", "This document is the second document.", "And this is the third one."]
summaries = ["Sample summary.", "Second summary.", "Third one."]

# Combine texts and summaries for vocabulary creation
all_texts = texts + summaries

# Tokenization function with punctuation removal
def tokenize(text):
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    return text.lower().split()

# Count token frequencies
token_counter = Counter(itertools.chain(*map(tokenize, all_texts)))

# Create the vocabulary with special tokens
special_tokens = ["<PAD>", "<UNK>"]
most_common_tokens = [token for token, _ in token_counter.most_common()]
vocab = {token: idx for idx, token in enumerate(special_tokens + most_common_tokens)}

# Optionally limit the vocabulary size
vocab_size = 10000  # Example vocabulary size
if len(vocab) > vocab_size:
    vocab = {k: v for k, v in itertools.islice(vocab.items(), vocab_size)}

vocab_inv = {idx: word for word, idx in vocab.items()}

# Define the updated simple tokenizer function
def simple_tokenizer(text, max_length, padding='max_length', truncation=True):
    tokens = tokenize(text)
    if truncation and len(tokens) > max_length:
        tokens = tokens[:max_length]
    token_ids = [vocab.get(token, vocab['<UNK>']) for token in tokens]
    if padding == 'max_length' and len(token_ids) < max_length:
        token_ids += [vocab['<PAD>']] * (max_length - len(token_ids))
    return token_ids

# Define a dataset class
class TextSummaryDataset(Dataset):
    def __init__(self, texts, summaries, tokenizer, max_len):
        self.texts = texts
        self.summaries = summaries
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        summary = self.summaries[idx]
        input_ids = self.tokenizer(text, max_length=self.max_len, padding='max_length', truncation=True)
        summary_ids = self.tokenizer(summary, max_length=self.max_len, padding='max_length', truncation=True)
        return torch.tensor(input_ids), torch.tensor(summary_ids)

# Example texts and summaries
texts = ["This is a sample document.", "This document is the second document.", "And this is the third one."]
summaries = ["Sample summary.", "Second summary.", "Third one."]

# Create dataset and dataloader
max_length = 10
dataset = TextSummaryDataset(texts, summaries, simple_tokenizer, max_length)
dataloader = DataLoader(dataset, batch_size=2)

# Define a basic transformer model
class TransformerBlock(nn.Module):
    def __init__(self, embed_size, heads, dropout):
        super(TransformerBlock, self).__init__()
        self.attention = nn.MultiheadAttention(embed_size, heads)
        self.norm1 = nn.LayerNorm(embed_size)
        self.norm2 = nn.LayerNorm(embed_size)
        self.ff = nn.Sequential(
            nn.Linear(embed_size, embed_size * 4),
            nn.ReLU(),
            nn.Linear(embed_size * 4, embed_size)
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        attn_output, _ = self.attention(x, x, x)
        x = self.norm1(attn_output + x)
        ff_output = self.ff(x)
        out = self.norm2(ff_output + x)
        return out

class SummarizerModel(nn.Module):
    def __init__(self, embed_size, heads, num_layers, vocab_size, max_len, dropout):
        super(SummarizerModel, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.pos_embedding = nn.Embedding(max_len, embed_size)
        self.layers = nn.ModuleList(
            [TransformerBlock(embed_size, heads, dropout) for _ in range(num_layers)]
        )
        self.fc_out = nn.Linear(embed_size, vocab_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        N, seq_length = x.shape
        positions = torch.arange(0, seq_length).expand(N, seq_length).to(x.device)
        out = self.dropout(self.embed(x) + self.pos_embedding(positions))
        for layer in self.layers:
            out = layer(out)
        out = self.fc_out(out)
        return out

# Hyperparameters
embed_size = 256
heads = 8
num_layers = 6
vocab_size = len(vocab)  # Use the actual vocab size
max_len = 10
dropout = 0.1

model = SummarizerModel(embed_size, heads, num_layers, vocab_size, max_len, dropout)

# Training loop
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)
criterion = nn.CrossEntropyLoss()

for epoch in range(3):  # Number of epochs
    model.train()
    for batch in dataloader:
        input_ids, summary_ids = batch
        optimizer.zero_grad()
        output = model(input_ids)
        loss = criterion(output.view(-1, vocab_size), summary_ids.view(-1))
        loss.backward()
        optimizer.step()
        print(f"Epoch: {epoch}, Loss: {loss.item()}")

# Inference function
def summarize(text, model, tokenizer, max_len=512):
    model.eval()
    input_ids = tokenizer(text, max_length=max_len, padding='max_length', truncation=True)
    input_ids = torch.tensor(input_ids).unsqueeze(0)  # Add batch dimension

    with torch.no_grad():
        output = model(input_ids)

    summary_ids = output.argmax(dim=-1).squeeze().tolist()
    summary = " ".join([vocab_inv[idx] for idx in summary_ids if idx not in [0]])  # Convert token IDs to words, ignoring padding
    return summary

# Example summarization
text = "This is a sample document."
summary = summarize(text, model, simple_tokenizer, max_len=10)
print(summary)


Epoch: 0, Loss: 3.017270565032959
Epoch: 0, Loss: 2.348728895187378
Epoch: 1, Loss: 1.6510303020477295
Epoch: 1, Loss: 1.2325745820999146
Epoch: 2, Loss: 0.8341148495674133
Epoch: 2, Loss: 0.6829085350036621



In [132]:
# Check tokenization
def print_tokens(text):
    tokens = tokenize(text)
    token_ids = [vocab.get(token, vocab['<UNK>']) for token in tokens]
    print(f"Tokens: {tokens}")
    print(f"Token IDs: {token_ids}")

print_tokens("This is a new document text to summarize.")


Tokens: ['this', 'is', 'a', 'new', 'document', 'text', 'to', 'summarize']
Token IDs: [2, 3, 11, 1, 4, 1, 1, 1]


In [133]:
print(f"Vocabulary Size: {len(vocab)}")
print(f"Sample Vocabulary: {list(vocab.items())[:10]}")


Vocabulary Size: 13
Sample Vocabulary: [('<PAD>', 0), ('<UNK>', 1), ('this', 2), ('is', 3), ('document', 4), ('sample', 5), ('the', 6), ('second', 7), ('third', 8), ('one', 9)]


In [134]:
def summarize(text, model, tokenizer, max_len=512):
    model.eval()
    input_ids = tokenizer(text, max_length=max_len, padding='max_length', truncation=True)
    input_ids = torch.tensor(input_ids).unsqueeze(0)  # Add batch dimension

    with torch.no_grad():
        output = model(input_ids)

    # Print output shape and values
    print(f"Model Output Shape: {output.shape}")
    print(f"Model Output: {output}")

    summary_ids = output.argmax(dim=-1).squeeze().tolist()
    print(f"Summary IDs: {summary_ids}")

    summary = " ".join([vocab_inv.get(idx, '<UNK>') for idx in summary_ids if idx not in [0]])  # Convert token IDs to words, ignoring padding
    return summary

# Example summarization
text = "This is a new document text to summarize."
summary = summarize(text, model, simple_tokenizer, max_len=10)
print(f"Generated Summary: {summary}")


Model Output Shape: torch.Size([1, 10, 13])
Model Output: tensor([[[ 1.4409e+00,  8.4210e-02, -8.8984e-01, -5.2063e-01, -4.7633e-01,
           3.0973e-01, -6.0323e-01,  8.8623e-01,  9.1380e-01,  4.5522e-02,
           3.4855e-01, -1.3336e+00, -5.2743e-01],
         [ 2.7349e+00, -5.0178e-01, -2.8367e-01, -7.8305e-01, -8.9332e-01,
          -2.7954e-02, -1.4764e-01,  4.2594e-03, -4.3328e-01,  3.4402e-01,
           9.9101e-01, -4.1014e-01, -6.2468e-01],
         [ 2.2964e+00,  8.5770e-02, -7.1061e-01,  3.8361e-02, -5.0908e-01,
           1.2301e-02, -6.7315e-01,  2.5079e-01, -1.2619e-02, -7.8412e-01,
           6.0353e-01, -3.9166e-01,  1.1125e-01],
         [ 3.4176e+00, -5.5115e-01, -3.3912e-01,  1.1945e-01, -6.9077e-01,
          -7.4563e-01, -7.0450e-02,  2.4930e-01,  3.1108e-02, -3.4038e-01,
          -1.3759e+00, -3.7956e-01, -1.0505e+00],
         [ 3.7560e+00,  2.6691e-01, -1.5133e-01, -1.0712e+00,  5.5558e-02,
           6.1040e-01,  3.2542e-01, -4.7108e-01, -1.1704e-01, -7.04

In [135]:
def simple_tokenizer(text, max_length, padding='max_length', truncation=True):
    tokens = tokenize(text)
    if truncation and len(tokens) > max_length:
        tokens = tokens[:max_length]
    token_ids = [vocab.get(token, vocab['<UNK>']) for token in tokens]
    if padding == 'max_length' and len(token_ids) < max_length:
        token_ids += [vocab['<PAD>']] * (max_length - len(token_ids))
    print(f"Token IDs (with padding/truncation): {token_ids}")
    return token_ids


In [136]:
def summarize(text, model, tokenizer, max_len=512):
    model.eval()
    input_ids = tokenizer(text, max_length=max_len, padding='max_length', truncation=True)
    input_ids = torch.tensor(input_ids).unsqueeze(0)  # Add batch dimension

    with torch.no_grad():
        output = model(input_ids)

    summary_ids = output.argmax(dim=-1).squeeze().tolist()
    print(f"Summary IDs: {summary_ids}")

    # Convert token IDs to words, ignoring padding
    summary = " ".join([vocab_inv.get(idx, '<UNK>') for idx in summary_ids if idx != vocab['<PAD>']])
    return summary
